# Geometric predicates

A colleague of mine once said, "Computational geometry is Satanically hard."
He wasn't joking.
Let's look at why.

The computational geometry equivalent of the "Hello world" program is to compute the convex hull of a set of points in 2D.
The simplest 2D convex hull algorithms proceed like so:
* Initialize the hull by picking two extreme points, say the points with the largest and smallest values of their $x$-coordinate respectively.
* For each edge $\{x, y\}$ of the current hull, and for every other point $z$ not known to be on the hull,
    + compute the *signed area* of the triangle $\{x, y, z\}$.
    + if all of the signed areas are positive, we're done with this edge.
    + if any points have negative signed area, find the point $z_*$ that gives the most negative area; remove $\{x, y\}$ from the hull and add the two edges $\{x, z_*\}$ and $\{z_*, y\}$.
 
Sounds easy enough.
Recall that the signed area of a set of points $\{x, y, z\}$ is the determinant of the following matrix:
$$\text{signed area}(x, y, z) = \det\left[\begin{matrix}1 & 1 & 1 \\ x_1 & y_1 & z_1 \\ x_2 & y_2 & z_2\end{matrix}\right]$$
When $x$, $y$, and $z$ are oriented counter-clockwise with respect to each other, the signed area is positive; when they are oriented clockwise, the signed area is negative, and when they are colinear, the signed area should be exactly equal to 0.
It's also worth pointing out that the signed area, mathematically, is invariant to cyclic permutations:
$$\text{signed area}(x, y, z) = \text{signed area}(y, z, x) = \text{signed area}(z, x, y).$$
Observe that the convex hull algorithm I wrote down above is branching on whether some signed areas are positive or negative.

**The hard part is what happens when three points are nearly co-linear.**
Using a naive approach to calculating the determinant, three positively-oriented but almost co-linear points can be incorrectly evaluated as being negatively-oriented.
Making this mistake can then lead to convex hulls that don't contain all their extreme points, or convex hulls that aren't even convex!

In the code below, I'll show some examples of how this basic geometric predicate can fail to be robust in floating-point arithmetic.

### Demonstration

First, I'm defining the determinant of a matrix in the most blunt way possible using the alternating-sum formula.
You might imagine instead calling a library function like np.linalg.det to do this instead.
Under the hood, numpy computes determinants by performing an LU factorization with partial pivoting, and then multiplying the diagonal elements of the U matrix.
This is perfectly reasonable but it can actually paper over the worst numerical instabilities some of the time, so for demonstrative purposes I'm doing this by hand.

In [ ]:
import numpy as np

def determinant(A):
    if A.shape == (2, 2):
        return A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]
    return sum(
        (-1) ** i * determinant(np.delete(A, i, axis=1)[1:, :])
        for i in range(A.shape[0])
    )

def naive_signed_area(xs):
    return determinant(np.vstack((np.ones(3), xs)))

The code below will take the coordinates of the three input points and perturb one of them (specified by `index`) in a grid of size `num_steps`.
Here we use the little-known (but very handy) function `nextafter`, which takes in a float and returns the next float.

In [ ]:
import math

def make_image(xs, index, num_steps, fn):
    image = np.zeros((2 * num_steps + 1, 2 * num_steps + 1))
    
    ys = np.copy(xs)
    for step in range(num_steps):
        ys[0, index] = math.nextafter(ys[0, index], -math.inf)
        ys[1, index] = math.nextafter(ys[1, index], -math.inf)
        
    zs = np.copy(ys)
    for row in range(2 * num_steps + 1):
        zs[0, index] = ys[0, index]
        for col in range(2 * num_steps + 1):
            zs[0, index] = math.nextafter(zs[0, index], math.inf)
            image[row, col] = fn(zs)
        zs[1, index] = math.nextafter(zs[1, index], math.inf)

    return image

Next, we need some points in the plane to test against.
We'll load some from a JSON file.
I generated these points in the first place at by making two points with random integer coordinates, defining a third point that lies along the line between the first two, generating the plots shown below, and looking one-by-one for interesting failure cases.
Using points with small integer coordinates guarantees that they can be represented exactly as floats and that the determinants should be exactly equal to 0 somewhere.

In [ ]:
import json

with open("points.json", "r") as input_file:
    points = np.array(json.load(input_file))

Next, we'll generate random samples of points in the plane to test against.
We first make two points at random, and then take the third point to lie along the line connecting the first two.
In order to be sure that they are really exactly colinear even in floating-point arithmetic, we use points with small integer values, which are exactly representable as floats.
In addition to testing perturbations of the first point, we also check to see what happens when we do a cyclic permutation of the inputs.

In [ ]:
fn = naive_signed_area

num_steps = 32
num_points = len(points)
images = np.zeros((num_points, 3, 2 * num_steps + 1, 2 * num_steps + 1))
for pindex, xs in enumerate(points):
    for index in range(3):
        zs = np.roll(xs, index, axis=1)
        images[pindex, index, :, :] = make_image(zs, index, num_steps, fn)

The plots below show the results of the naively-implemented signed area predicate when we perturb the 1st point.
The first column shows the results with the point set as given; the next column shows the results when we cyclically permute the points once; and the final column shows the results of cyclically permuting twice.
Despite the fact that the signed area predicate is unchanged by cyclic permutations in real arithmetic, the results are not the same in floating-point arithmetic.

In [ ]:
import matplotlib.pyplot as plt

figsize = (6, 2 * num_points)
fig, axes = plt.subplots(
    nrows=num_points, ncols=3, sharex=True, sharey=True, figsize=figsize
)
for ax in axes.flatten():
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

for index in range(num_points):
    for ax, image in zip(axes[index, :], images[index, :, :, :]):
        ax.imshow(np.sign(image), cmap="BrBG", origin="lower")

### Solution

Many higher-level geometric algorithms -- convex hulls, Delaunay triangulations -- have to branch on the output of a low-level geometric predicate that is inherently unstable in floating-point arithmetic.
How do we dig ourselves out of this hole?

In the 90s, Douglas Priest, Jonathan Richard Shewchuk, and others developed an approach based on implementing higher-precision floating-point arithmetic.
Using some very careful and painstaking analysis of the computational kernels, they were able to determine ahead of time how to detect when one is close to an edge case and what is the maximum precision necessary.
Shewchuk's original code has found its way into tons of software packages almost unmodified.
This approach has demonstrably been very effective, but it's hard to understand except for real experts.

As an alternative, we could exactly convert all floating-point values to [bignum](https://en.wikipedia.org/wiki/Arbitrary-precision_arithmetic) rationals and do all the arithmetic exactly.
This approach is robust but more expensive by several orders of magnitude.

We can combine the two ideas by first using [interval arithmetic](https://en.wikipedia.org/wiki/Interval_arithmetic) to compute the determinant.
If the result interval does not contain zero, then we can use the median as the signed area; the sign is sure to be correct.
If the result interval does contain zero, we fall back to bignum rational arithmetic.
I took this approach in writing the predicates library; Boost provides both interval and bignum rational arithmetic.
The code below reproduces the figure above but using the determinant computation from predicates.

In [ ]:
import predicates

def exact_signed_area(xs):
    return predicates.orientation(xs)

fn = exact_signed_area
images = np.zeros((num_points, 3, 2 * num_steps + 1, 2 * num_steps + 1))
for pindex, xs in enumerate(points):
    for index in range(3):
        zs = np.roll(xs, index, axis=1)
        images[pindex, index, :, :] = make_image(zs, index, num_steps, fn)

You can see below that the region of zero signed area is confined to a single thin stripe and the positive, negative, and zero regions are all connected.
Moreover, the answers are the same even under cyclic permutations.

In [ ]:
figsize = (6, 2 * num_points)
fig, axes = plt.subplots(
    nrows=num_points, ncols=3, sharex=True, sharey=True, figsize=figsize
)
for ax in axes.flatten():
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

for index in range(num_points):
    for ax, image in zip(axes[index, :], images[index, :, :, :]):
        ax.imshow(np.sign(image), cmap="BrBG", origin="lower")